In [ ]:
drugs= ['25i-nbome', '2c-b', '2c-c', '2c-d', '2c-e', '2c-i', '2c-p', '2c-t-2', '2c-t-4', '2c-t-7',
            '5-meo-dmt', '5-meo-dipt', '5-meo-mipt', '5-meo-tmt', 'amphetamine', 'alprazolam',
            'diphenhydramine', 'dipt', 'dpt',  'dmt', 'dob', 'doi', 'dom', 'dxm', 'ketamine', 'lsd', 'mda', 'mdma', 'mescaline',
            'methamphetamine', 'morphine', 'pcp', 'psilocin', 'salvia', 'tma-2', 'aripiprazole',
            'bupropion', 'thc', 'cocaine', 'haloperidol', 'hydrocodone', 'hydromorphone', 'ibogaine',
            'methadone', 'mirtazapine', 'olanzapine', 'oxycodone',
            'paroxetine', 'quetiapine', 'risperidone', 'sertraline', 'venlafaxine']

receptors = ['5-HT2A', '5-HT2C', '5-HT2B', '5-HT1A', '5-HT1B', '5-HT1D', '5-HT1E', '5-HT1F', '5-HT3',
                 '5-HT5A', '5-HT6', '5-HT7', 'DOPAMINE D1', 'DOPAMINE D2', 'DOPAMINE D3', 'DOPAMINE D4', 'DOPAMINE D5',
                 'Adrenergic Alpha1A', 'Adrenergic Alpha1B', 'Adrenergic Alpha2A', 'Adrenergic Alpha2B', 'Adrenergic Beta1', 'Adrenergic Beta2',
                 'SERT', 'DAT', 'NET',
                 'Imidazoline 1', 'Sigma 1', 'Sigma 2', 'DOR', 'KOR', 'MOR', 'M1', 'M2', 'M3', 'M4', 'M5', 'H1', 'H2', 'H3', 'H4',
                 'Calcium Channel', 'NMDA', 'CB1', 'CB2', 'Glutamate-AMPA', 'GABA A', 'GABA B', 'DOPAMINE D2 Long', 'DOPAMINE D2 Short', 'Sodium Channel',
                 'TAAR1', 'Substance P', 'PAF Platelet activating factor', 'Prostaglandin E3',
                 'Prostaglandin E4',  'HERG', 'Monoamine Oxidase A', 'Monoamine Oxidase B', 'Cholecystokinin A', 'Cholecystokinin B',]
print(f'd:{len(drugs)} r:{len(receptors)}')

In [ ]:
!pip install tensorflow-text==2.5
!pip install tf-models-official==2.5

In [ ]:
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

import matplotlib.pyplot as plt
from ml4h.TensorMap import TensorMap, Interpretation

drug_folder = 'split_drugs'
drug_folder = 'split_small_test_all'

preprocess_model = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

#base_model = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
base_model = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2"

tf.get_logger().setLevel('ERROR')

In [ ]:
df = pd.read_csv(f'./all_mask_v2022_04_15_meta_data.csv')
# df = df[df.age != 'Not Given']
# df = df[df.sex_int.notna()]
df = df[df.text.notna()]

df.psychoactive_class = df.psychoactive_class.apply(lambda x: x.strip())

drug2class = {d:i for i,d in enumerate(df.drug.unique())}
psychoactive2class = {d:i for i,d in enumerate(df.psychoactive_class.unique())}
class2weight = {i:5*(len(df)/len(df[df.drug==d])) for i,d in enumerate(df.drug.unique())}
print(psychoactive2class)
df['drug_class'] = [drug2class[d] for d in df.drug]
df['psychoactive_class_int'] = [psychoactive2class[d] for d in df.psychoactive_class]
df[[f'tag_{i}' for i in range(52) ]] = df[[f'tag_{i}' for i in range(52) ]].fillna(0)


# train = df.sample(frac = 0.8)
# test = df.drop(train.index).sample(frac = 0.5)
# validate = df.drop(train.index).drop(test.index)


tags = {'Small_Group': 'tag_0', 'General': 'tag_1', 'First_Times': 'tag_2', 'Alone': 'tag_3', 'Difficult_Experiences': 'tag_4', 'Glowing_Experiences': 'tag_5', 'Retrospective_Summary': 'tag_6', 'Various': 'tag_7', 'Unknown_Context': 'tag_8', 'Mystical_Experiences': 'tag_9', 'Health_Problems': 'tag_10', 'Combinations': 'tag_11', 'Not_Applicable': 'tag_12', 'Bad_Trips': 'tag_13', 'Hangover_Days_After': 'tag_14', 'Entities_Beings': 'tag_15', 'Music_Discussion': 'tag_16', 'Addiction_Habituation': 'tag_17', 'Post_Trip_Problems': 'tag_18', 'Nature_Outdoors': 'tag_19', 'Relationships': 'tag_20', 'Depression': 'tag_21', 'Therapeutic_Intent_or_Outcome': 'tag_22', 'Overdose': 'tag_23', 'Medical_Use': 'tag_24', 'Sex_Discussion': 'tag_25', 'Train_Wrecks_Trip_Disasters': 'tag_26', 'Guides_Sitters': 'tag_27', 'Rave_Dance_Event': 'tag_28', 'Preparation_Recipes': 'tag_29', 'Festival_Lg_Crowd': 'tag_30', 'Health_Benefits': 'tag_31', 'Large_Group': 'tag_32', 'Multi-Day_Experience': 'tag_33', 'Club_Bar': 'tag_34', 'What_Was_in_That': 'tag_35', 'Personal_Preparation': 'tag_36', 'HPPD_Lasting_Visuals': 'tag_37', 'Families': 'tag_38', 'Second_Hand_Report': 'tag_39', 'Loss_of_Magic': 'tag_40', 'Hospital': 'tag_41', 'Public_Space': 'tag_42', 'School': 'tag_43', 'Poetry': 'tag_44', 'Performance_Enhancement': 'tag_45', 'Large_Party': 'tag_46', 'Group_Ceremony': 'tag_47', 'Workplace': 'tag_48', 'Cultivation_Synthesis': 'tag_49', 'Pregnancy_Baby': 'tag_50', 'Military': 'tag_51'}
itags = {v: k.replace('(', '').replace(')', '').replace('/', '') for k, v in tags.items()}
ctags = {v: int(k.replace('tag_', '')) for k, v in itags.items()}

receptors = ['5_ht2a', '5_ht2c', '5_ht2b', '5_ht1a', '5_ht1b', '5_ht1d', '5_ht1e', '5_ht1f', '5_ht3', '5_ht5a', '5_ht6', '5_ht7', 'dopamine_d1', 'dopamine_d2', 'dopamine_d3', 'dopamine_d4', 'dopamine_d5', 'adrenergic_alpha1a', 'adrenergic_alpha1b', 'adrenergic_alpha2a', 'adrenergic_alpha2b', 'adrenergic_beta1', 'adrenergic_beta2', 'sert', 'dat', 'net', 'imidazoline_1', 'sigma_1', 'sigma_2', 'dor', 'kor', 'mor', 'm1', 'm2', 'm3', 'm4', 'm5', 'h1', 'h2', 'h3', 'h4', 'calcium_channel', 'nmda', 'cb1', 'cb2', 'glutamate_ampa', 'gaba_a', 'gaba_b', 'dopamine_d2_long', 'dopamine_d2_short', 'sodium_channel', 'taar1', 'substance_p', 'paf_platelet_activating_factor', 'prostaglandin_e3', 'prostaglandin_e4', 'herg', 'monoamine_oxidase_a', 'monoamine_oxidase_b', 'cholecystokinin_a', 'cholecystokinin_b']
df[receptors] = df[receptors].astype(float)

In [ ]:
df.info()

In [ ]:
df['5_ht2a'].plot.hist(bins=40)

In [ ]:
for r in receptors:
    df[r] -= df[r].mean()
    df[r] /= df[r].std()
    df[r] *= 10

In [ ]:
receptors2learn = []
for r in receptors:
    if len(df[r].value_counts()) > 10:
        receptors2learn.append(r)
        df[r].plot.hist()

In [ ]:
df['no_tags'] = df[[f'tag_{i}' for i in range(52) ]].sum(axis=1)
print(df['no_tags'].value_counts())

In [ ]:
df.tag_0.value_counts()

In [ ]:
df[df.set=='train'].to_csv('train.csv', index=False)
df[df.set=='valid'].to_csv('valid.csv', index=False)
df[df.set=='test'].to_csv('test.csv', index=False)

In [ ]:
#output_cols = [ 'drug_class', 'psychoactive_class'] + receptors2learn
output_cols = [ 'drug_class']
#output_cols += [f'tag_{i}' for i in range(52)]
tensor_maps_out = []
for oc in output_cols:
    if 'drug_class' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,),
                                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'drug_{d}': v for d,v in drug2class.items()}))
    elif 'psychoactive_class_int' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,),
                                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'{d}': v for d,v in psychoactive2class.items()}))        
    elif 'tag_' in oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,), 
                                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'no_{itags[oc]}': 0, f'{itags[oc]}': 1}))
    elif 'age' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CONTINUOUS, shape=(1,),
                                         loss=tf.keras.losses.LogCosh(),
                                         metrics=[tf.metrics.MeanAbsoluteError()],))
    elif oc in receptors:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CONTINUOUS, shape=(1,),
                                         loss=tf.keras.losses.MeanSquaredError(),
                                         metrics=[tf.metrics.MeanAbsoluteError()],))
    else:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,), 
                                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'no_{oc}':0, f'{oc}':1}))
        
def make_dataset(csv, in_cols, out_cols, batch_size=32):
    i = tf.data.experimental.make_csv_dataset(csv, select_columns=in_cols, 
                                              batch_size=1, shuffle=False)
    o = tf.data.experimental.make_csv_dataset(csv, select_columns=out_cols, 
                                              batch_size=1, shuffle=False)
    ds = tf.data.Dataset.zip((i,o))
    ds = ds.shuffle(10000)
    ds = ds.unbatch().batch(batch_size)
    
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

train_ds = make_dataset('train.csv', ['text'], output_cols)
valid_ds = make_dataset('valid.csv', ['text'], output_cols)
test_ds = make_dataset('test.csv', ['text'], output_cols)

In [ ]:
from ml4h.TensorMap import TensorMap, Interpretation

In [ ]:
for feature_batch, label in test_ds.take(1):
    print(f"label {label}")
    for key, value in feature_batch.items():
        print(f"\n\n\n Key is  {key:20s}: {value[0]}")

In [ ]:
n_drugs = len(drug2class)
dropout_rate = 0.2

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
bert_preprocess_model = hub.KerasLayer(
    preprocess_model)

bert_model = hub.KerasLayer(
    base_model,
    trainable=True)

text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

bert_results = bert_model(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

def weighted_scce(weights):
    def my_loss(y_true, y_pred):
        sample_weights = [weights[int(y_true[i].numpy())] for i in 33]
        scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        return scce(y_true, y_pred, sample_weights=sample_weights)
    return my_loss

def build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder, tensor_maps_out):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(dropout_rate)(net)
    #net = tf.keras.layers.Dense(256, activation='swish')(net)
    #net = tf.keras.layers.Dropout(dropout_rate)(net)
    outputs = []
    metrics = []
    losses = []    
    for otm in tensor_maps_out:
        if otm.is_categorical():
            outputs.append(tf.keras.layers.Dense(len(otm.channel_map), activation=None, name=otm.name)(net))
            losses.append(tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
            metrics.append(tf.metrics.SparseCategoricalAccuracy(name=f'{otm.name}_SparseCategoricalAccuracy_met'))
        elif otm.is_continuous():
            netl = tf.keras.layers.Dense(otm.annotation_units, activation='swish')(net)
            netl = tf.keras.layers.Dropout(dropout_rate)(netl)         
            outputs.append(tf.keras.layers.Dense(1, activation='linear', name=otm.name)(net))
            losses.append(tf.keras.losses.MeanSquaredError()) 
            metrics.append(tf.metrics.MeanAbsoluteError(name=f'{otm.name}_mae'))
    return tf.keras.Model(text_input, outputs), losses, metrics

classifier_model, losses, metrics = build_classifier_model(preprocess_model, base_model, tensor_maps_out)
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result[-1]))
tf.keras.utils.plot_model(classifier_model)
metrics = {tm.name: tm.metrics for tm in tensor_maps_out}
losses = [tm.loss for tm in tensor_maps_out]
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#metrics = tf.metrics.SparseCategoricalAccuracy()

In [ ]:
epochs = 12
batch_size = 32
steps_per_epoch = len(df[df.set=='train'])//batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
print(f'warm up {steps_per_epoch}  and {num_train_steps}')
init_lr = 1e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


classifier_model.compile(optimizer=optimizer, loss=losses, metrics=metrics)

In [ ]:
print(f'Training model with bert')
history = classifier_model.fit(x=train_ds, steps_per_epoch=steps_per_epoch, 
                               validation_data=valid_ds, validation_steps=len(df[df.set=='valid'])//batch_size,
                               epochs=epochs, shuffle=True
                              )

In [ ]:
loss = classifier_model.evaluate(test_ds, steps=len(df[df.set=='test'])//batch_size)

print(f'Loss: {loss}')

In [ ]:
from ml4h.plots import plot_metric_history
plot_metric_history(history, steps_per_epoch, 'Receptor Learning Curves')

In [ ]:
classifier_model.summary()

In [ ]:
#classifier_model.save('bert_48_drug_sex_age_classifier')
#classifier_model.save('bert_48_drug_sex_52_tag_classifier')

In [ ]:
# classifier_model = tf.keras.models.load_model(f'bert_48_drug_sex_age_classifier', 
#                                               custom_objects={'AdamWeightDecay':optimizer})
classifier_model = tf.keras.models.load_model(f'bert_48_drug_sex_52_tag_classifier', 
                                              custom_objects={'AdamWeightDecay':optimizer})

In [ ]:
len(df[df.set=='test'])

In [ ]:
from collections import defaultdict
predictions = defaultdict(list)
truths = defaultdict(list)
for text, labels in test_ds.as_numpy_iterator():
    for l in labels:
        truths[l].extend(map(int, list(labels[l])))
    p = classifier_model.predict(text)
    if len(classifier_model.output_names) == 1:
        p = [p]
    for i,ot in enumerate(classifier_model.output_names):
        predictions[ot].extend(list(p[i]))
        
    if len(truths[l]) >= len(df[df.set=='test']):
        break

In [ ]:
from ml4h.plots import plot_roc, subplot_rocs, plot_scatter
def make_one_hot(y, num_labels):
    ohy = np.zeros((y.shape[-1], num_labels))
    for i in range(0, y.shape[-1]):
        ohy[i, int(y[i])] = 1.0
    return ohy

rocs = []
perfs = {}
for otm in tensor_maps_out:
    if otm.is_categorical():
        print(f' otm {otm} {np.array(predictions[otm.name]).shape}')
        perfs[otm] = plot_roc(np.array(predictions[otm.name]), 
                 make_one_hot(np.array(truths[otm.name]), len(otm.channel_map)), 
                 otm.channel_map, otm.name)
        rocs.append((np.array(predictions[otm.name]), 
                     make_one_hot(np.array(truths[otm.name]), len(otm.channel_map)), 
                     otm.channel_map))
    elif otm.is_continuous():
        plot_scatter(np.array(predictions[otm.name]), np.array(truths[otm.name]), otm.name)
subplot_rocs(rocs)

In [ ]:
tm = tensor_maps_out[0]
pmax = np.argmax(np.array(predictions[tm.name]), axis=-1)

pmax.shape

confusion = np.zeros((len(tm.channel_map), len(tm.channel_map)), dtype=np.int32)
for i in range(len(tm.channel_map)):
    for j in range(len(tm.channel_map)):
        confusion[i,j] += sum((pmax==j) * (np.array(truths[tm.name]) == i))
        

# pmax == 24

# truths[tm.name] == 24

# confusion

# np.array(truths[tm.name]) == 24

import seaborn as sb
%matplotlib inline
import matplotlib.pyplot as plt
from io import StringIO

def confusion_heatmap(confusion, cutoff=0.001, fmt='2d'):
    fig, ax = plt.subplots(figsize=(24, 12), dpi=300)
    ax = sb.heatmap(confusion, cmap=sb.diverging_palette(220, 20, as_cmap=True), ax=ax, cbar=False)
    ax = sb.heatmap(confusion, mask=confusion < cutoff, cmap=sb.diverging_palette(220, 20, as_cmap=True), 
                    annot=True, fmt=fmt, cbar_kws={"shrink": .8, 'label': 'Counts'}, ax=ax)

    ax.set_title('Drug Confusion Matrix')
    ax.set_xlabel('Drugs')
    ax.set_ylabel('Drugs')
    ax.set_xticks(np.arange(confusion.shape[0]) + 0.5)
    ax.set_yticks(np.arange(confusion.shape[0]) + 0.5)
    # update the desired text annotations
#     for text in ax.texts:
#         if text.get_text() == '0':
#             text.set_text('.')
    ax.set_xticklabels(labels=list(tm.channel_map.keys()), ha='right', rotation=30)
    ax.set_yticklabels(labels=list(tm.channel_map.keys()), rotation=0)
    plt.tight_layout()
    figure_path='./confusion_heatmap.png'
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    plt.show()
    
confusion_heatmap(confusion)    

In [ ]:
confusion_r = np.zeros((len(tm.channel_map), len(tm.channel_map)))
for i in range(len(tm.channel_map)):
    for j in range(len(tm.channel_map)):
        confusion_r[i,j] += sum((pmax==j) * (np.array(truths[tm.name]) == i)) / sum(np.array(truths[tm.name]) == i)

confusion_heatmap(confusion_r, fmt=".2f")  

In [ ]:
len(test['drug_class'].value_counts())